# 图像-视觉多模态理解模型 (VLM with image-input, Image-text-to-text)

图像文本到文本模型，也称为视觉语言模型（VLMs），是接受图像输入的语言模型。这些模型可以处理各种任务，从视觉问答到图像分割。这个任务与图像到文本有许多相似之处，但也有一些重叠的用例，如图像描述。图像到文本模型只接受图像输入，通常完成一个特定的任务，而VLMs接受开放式的文本和图像输入，是更通用的模型。

在本指南中，我们将简要概述VLMs，并展示如何使用Transformers进行推理。

首先，有几种类型的VLMs：

- 用于微调的基础模型
- 用于对话的聊天微调模型
- 用于指令的微调模型

本指南重点介绍使用指令微调模型进行推理。

让我们开始安装依赖项。

In [ ]:
pip install -q transformers accelerate flash_attn

现在初始化模型和处理器。

In [ ]:
from transformers import AutoProcessor, AutoModelForImageTextToText
import torch

device = torch.device("cuda")
model = AutoModelForImageTextToText.from_pretrained(
    "HuggingFaceM4/idefics2-8b",
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
).to(device)

processor = AutoProcessor.from_pretrained("HuggingFaceM4/idefics2-8b")

这个模型有一个聊天模板，帮助用户解析聊天输出。此外，该模型还可以在单个对话或消息中接受多个图像作为输入。我们将准备输入图像，如下所示：

![两只猫坐在网上](../../resources/images/cats.png)

![一只蜜蜂在粉红色的花朵上](../../resources/images/bee.jpg)

下面是聊天模板的示例。我们可以通过在模板末尾附加它来提供对话回合和最后一条消息作为输入。

In [ ]:
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": "What do we see in this image?"}
        ]
    },
    {
        "role": "assistant",
        "content": [
            {"type": "text", "text": "In this image we can see two cats on the nets."}
        ]
    },
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": "And how about this image?"}
        ]
    },
]

我们现在将调用处理器的`apply_chat_template()`方法来预处理其输出以及图像输入。

In [ ]:
prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(text=prompt, images=[images[0], images[1]], return_tensors="pt").to(device)

我们现在可以将预处理后的输入传递给模型。

In [ ]:
with torch.no_grad():
    generated_ids = model.generate(**inputs, max_new_tokens=500)
generated_texts = processor.batch_decode(generated_ids, skip_special_tokens=True)

print(generated_texts)

## 流式传输

我们可以使用文本流式传输来获得更好的生成体验。Transformers支持使用`TextStreamer`或`TextIteratorStreamer`类进行流式传输。我们将使用`TextIteratorStreamer`与`IDEFICS-8B`。

假设我们有一个应用程序，它保留聊天历史并接收新的用户输入。我们将像往常一样预处理输入，并初始化`TextIteratorStreamer`来在单独的线程中处理生成。这允许你实时流式传输生成的文本标记。任何生成参数都可以传递给`TextIteratorStreamer`。

In [ ]:
import time
from transformers import TextIteratorStreamer
from threading import Thread

def model_inference(
    user_prompt,
    chat_history,
    max_new_tokens,
    images
):
    user_prompt = {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": user_prompt},
        ]
    }
    chat_history.append(user_prompt)
    streamer = TextIteratorStreamer(
        processor.tokenizer,
        skip_prompt=True,
        timeout=5.0,
    )

    generation_args = {
        "max_new_tokens": max_new_tokens,
        "streamer": streamer,
        "do_sample": False
    }

    # add_generation_prompt=True使模型生成机器人响应
    prompt = processor.apply_chat_template(chat_history, add_generation_prompt=True)
    inputs = processor(
        text=prompt,
        images=images,
        return_tensors="pt",
    ).to(device)
    generation_args.update(inputs)

    thread = Thread(
        target=model.generate,
        kwargs=generation_args,
    )
    thread.start()

    acc_text = ""
    for text_token in streamer:
        time.sleep(0.04)
        acc_text += text_token
        if acc_text.endswith("<end_of_utterance>"):
            acc_text = acc_text[:-18]
        yield acc_text

    thread.join()

现在让我们调用我们创建的`model_inference`函数并流式传输值。

In [ ]:
generator = model_inference(
    user_prompt="And what is in this image?",
    chat_history=messages,
    max_new_tokens=100,
    images=images
)

for value in generator:
    print(value)

## 在较小硬件上适配模型

VLMs通常很大，需要优化以适应较小的硬件。Transformers支持许多模型量化库，这里我们只展示使用Quanto的int8量化。int8量化提供了高达75％的内存改进（如果所有权重都被量化）。然而，它并不是免费的午餐，因为8位不是CUDA本机精度，权重在运行时被量化来回转换，这增加了延迟。

首先，安装依赖项。

In [ ]:
pip install -U quanto bitsandbytes

要在加载期间量化模型，我们首先需要创建`QuantoConfig`。然后像往常一样加载模型，但在模型初始化期间传递`quantization_config`。

In [ ]:
from transformers import AutoModelForImageTextToText, QuantoConfig

model_id = "HuggingFaceM4/idefics2-8b"
quantization_config = QuantoConfig(weights="int8")
quantized_model = AutoModelForImageTextToText.from_pretrained(
    model_id, device_map="cuda", quantization_config=quantization_config
)

就这样，我们可以使用模型，无需任何更改。

## 进一步阅读

这里有一些关于图像文本到文本任务的更多资源。

- [图像文本到文本任务页面](https://huggingface.co/tasks/image-text-to-text)涵盖了模型类型、用例、数据集等。
- [视觉语言模型解释](https://huggingface.co/blog/vlms)是一篇博客文章，涵盖了关于视觉语言模型和使用TRL进行监督微调的所有内容。